#Matrix-matrix multiplication
First we will look in to the serial code given below.


In [1]:
%%writefile matmatserial.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
int n=640;
int matmul(int a[n][n], int b[n][n],int c[n][n], int n)
{ int sum=0;int i,j;int k=0;
  for(int i=0;i<n;++i)
  {sum=0;
    for(int j=0;j<n;++j)
    {
      for(int h=0;h<n;h++)
      {
        sum=sum+a[i][h]*b[h][j];
      }
      c[i][j]=sum;
    }

  }
}
void main()
{ double total_time=0.000;
	clock_t begin=clock();
  int i,j;

  int a[n][n]; int b[n][n]; int c[n][n];
  for(int i=0;i<n;i++)
  {
    for(int j=0;j<n;j++)
  {
    a[i][j]=i+1;
    b[i][j]=i+1;
    c[i][j]=0;
  }
}
matmul(a,b,c,n);
/*for(i=0;i<n;i++)
{
  for( j=0;j<n;j++)
{

printf("%d " ,c[i][j]);
}printf("\n");
}*//*free (a);
free(b);
free(c);*/
clock_t end =clock();
total_time += ((double) (end - begin)) / CLOCKS_PER_SEC;
printf(" time %lf\n",total_time*1000 );
}


Writing matmatserial.c


In [3]:
!gcc matmatserial.c -o matmatserial

In [5]:
!./matmatserial

 time 1067.403000


Note down the time taken on CPU version of code.
On to the GPU version!!!

Changing the runtime and installing the nvidia compilers next.

In [9]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter.plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s4e7yha9
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s4e7yha9
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-ddm5fm_f/wheels/7d/b9/66/459b9938664e6a93d1a85323ec52f7e51cd7265d253410a7d8
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpau9xhao2".


In [2]:
%%writefile matmatparallel.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <time.h>
int n=640;

__global__ void paramul(int *ad, int *bd, int *cd,int n)
{
  int k;/*printf ("hi");*/
  int idx=blockIdx.x*blockDim.x+threadIdx.x;
  int idy=blockIdx.y*blockDim.y+threadIdx.y;
  //printf ("%d", idx);
  if(idx<n && idy<n)
  {
    for(k=0;k<n;k++)
    (*(cd+n*idx+idy))=(*(cd+n*idx+idy))+(*(ad+n*idx+k))*(*(bd+n*k+idy));
  }
__syncthreads();
}

void matmul(int*ah,int*bh,int*ch)
{
  int*ad;int*bd;int*cd;
  int size=n*n*sizeof(int);
  cudaMalloc((void**)&ad,size);
  cudaMalloc((void**)&bd,size);
  cudaMalloc((void**)&cd,size);
  cudaMemcpy(ad,ah,size,cudaMemcpyHostToDevice);
  cudaMemcpy(bd,bh,size,cudaMemcpyHostToDevice);
  int g;
  int d=8;
  g=ceil(n/d);
  dim3 dimGrid (g,g);
  dim3 dimBlock (d,d);
//  printf("ho");

  paramul <<<dimGrid,dimBlock>>>(ad,bd,cd,n);

  cudaMemcpy(ch,cd,size,cudaMemcpyDeviceToHost);
  cudaFree(ad);
  cudaFree(bd);
  cudaFree(cd);

}

int main ()
{
  double total_time=0.000;
  	clock_t begin=clock();
  int *ah;int*bh;int*ch;int i,j;
  ah=(int*)malloc(n*n*sizeof(int));
  bh=(int*)malloc(n*n*sizeof(int));
  ch=(int*)malloc(n*n*sizeof(int));
  for(i=0;i<n;i++)
  {
    for(j=0;j<n;j++)
    {
      *(ah+n*i+j)=i+1;
      *(bh+n*i+j)=1;
      *(ch+n*i+j)=0;
    }
  }
matmul(ah,bh,ch);
/*for(i=0;i<n;i++)
{
  for(j=0;j<n;j++)
  { printf ("%d ",(*(ch+n*i+j)));
  }
  printf("\n");
}
*/

free(ah);
free(bh);
free(ch);
clock_t end =clock();
total_time += ((double) (end - begin)) / CLOCKS_PER_SEC;
printf(" time %lf\n",total_time*1000 );

}


Writing matmatparallel.cu


In [2]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 matmatparallel.cu -o matmatparallel

In [3]:
!./matmatparallel

 time 218.273000


Can we leverage the shared memory??
Yes!! Let's see how.

In [3]:
%%writefile matmatshared.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <time.h>
int n=640;

__global__ void paramul(int *ad, int *bd, int *cd,int n)
{
  int k;int tile=16;int m;
  int idx=blockIdx.x*blockDim.x+threadIdx.x;
  int idy=blockIdx.y*blockDim.y+threadIdx.y;
 int bx=blockIdx.x;int by=blockIdx.y;
  int tx=threadIdx.x; int ty=threadIdx.y;
  int row = by *blockDim.y + ty;
 int col = bx *blockDim.x + tx;
 __shared__ int as[16][16];
 __shared__ int bs[16][16];
  if(idx<n && idy<n)
  {
    for(m=0;m<n/tile;++m)
    {as[ty][tx]=*(ad+(row*n+(m*tile)+tx));
    bs[ty][tx]=*(bd+((m*tile+ty)*n+col));
    __syncthreads();
    for (k=0;k<tile;++k)
    cd[row*n + col]+=as[ty][k]*bs[k][tx];
  }
}
__syncthreads();
}

void matmul(int*ah,int*bh,int*ch)
{
  int*ad;int*bd;int*cd;
  int size=n*n*sizeof(int);
  cudaMalloc((void**)&ad,size);
  cudaMalloc((void**)&bd,size);
  cudaMalloc((void**)&cd,size);
  cudaMemcpy(ad,ah,size,cudaMemcpyHostToDevice);
  cudaMemcpy(bd,bh,size,cudaMemcpyHostToDevice);
  int g;
  int d=16;
  g=ceil(n/d);
  dim3 dimGrid (g,g);
  dim3 dimBlock (d,d);
//  printf("ho");

  paramul <<<dimGrid,dimBlock>>>(ad,bd,cd,n);

  cudaMemcpy(ch,cd,size,cudaMemcpyDeviceToHost);
  cudaFree(ad);
  cudaFree(bd);
  cudaFree(cd);

}

int main ()
{ double total_time=0.000;
	clock_t begin=clock();
  int *ah;int*bh;int*ch;int i,j;
  ah=(int*)malloc(n*n*sizeof(int));
  bh=(int*)malloc(n*n*sizeof(int));
  ch=(int*)malloc(n*n*sizeof(int));
  for(i=0;i<n;i++)
  {
    for(j=0;j<n;j++)
    {
      *(ah+n*i+j)=i+1;
      *(bh+n*i+j)=1;
      *(ch+n*i+j)=0;
    }
  }
matmul(ah,bh,ch);
/*for(i=0;i<n;i++)
{
  for(j=0;j<n;j++)
  { printf ("%d ",(*(ch+n*i+j)));
  }
  printf("\n");
}
*/

free(ah);
free(bh);
free(ch);
clock_t end =clock();
total_time += ((double) (end - begin)) / CLOCKS_PER_SEC;
printf(" time %lf\n",total_time*1000 );

}


Writing matmatshared.cu


In [5]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 matmatshared.cu -o matmatshared

In [6]:
!./matmatshared

 time 215.325000
